In [23]:
## Packages and Options
import html
import saspy
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
sas=saspy.SASsession()

Using SAS Config named: httpsviya


/opt/conda/lib/python3.10/site-packages/saspy/sasiohttp.py:914: UserWarning: Noticed 'ERROR:' in LOG, you ought to take a look and see if there was a problem
  warnings.warn("Noticed 'ERROR:' in LOG, you ought to take a look and see if there was a problem")


SAS server started using Context SAS Studio compute context with SESSION_ID=55d10860-5b8d-45a1-9fdf-4d34c955e10c-ses0000


In [24]:
## Access Data
df_raw = pd.read_csv(r'https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/home_equity.csv')

In [25]:
## Prepare Data
df = (df_raw
      .fillna(df_raw[df_raw.select_dtypes(include = np.number).columns.to_list()].mean())         ## Fill all numeric missing values with the mean
      .fillna(df_raw[df_raw.select_dtypes(include = object).columns.to_list()].mode().iloc[0])    ## Fill all character missing values with the mode
      .assign(DIFF = lambda _df: _df.MORTDUE - _df.VALUE,                                         ## Difference between mortgage due and value
              LOAN_STATUS = lambda _df: _df.BAD.map({1:'Default', 0:'Repaid'})                        ## Map values of 1 and 0 with the values Default and Repaid        
       )
      .rename(columns=lambda colName:colName.lower().replace("_",""))                             ## Lowercase column names and remove underscores
)

In [26]:
## Preview the dataframe and number of missing values
print(df.head(5))
print(df.isna().sum())

   bad  loan     mortdue          value   reason     job        yoj    derog  \
0    1  1100  25860.0000   39025.000000  HomeImp   Other  10.500000  0.00000   
1    1  1300  70053.0000   68400.000000  HomeImp   Other   7.000000  0.00000   
2    1  1500  13500.0000   16700.000000  HomeImp   Other   4.000000  0.00000   
3    1  1500  73760.8172  101776.048741  DebtCon   Other   8.922268  0.25457   
4    0  1700  97800.0000  112000.000000  HomeImp  Office   3.000000  0.00000   

     delinq       clage      ninq       clno    debtinc  appdate        city  \
0  0.000000   94.366667  1.000000   9.000000  33.779915    22040      oregon   
1  2.000000  121.833333  0.000000  14.000000  33.779915    22229   churchton   
2  0.000000  149.466667  1.000000  10.000000  33.779915    21396       orcas   
3  0.449442  179.766275  1.186055  21.296096  33.779915    22213    hastings   
4  0.000000   93.333333  0.000000  14.000000  33.779915    22329  wilmington   

        state            division   re

In [27]:
##
## Load the DataFrame to different locations in SAS Viya
##

# Load the DataFrame to the compute server as a SAS data set
sas.df2sd(df, 'home_equity_compute_py')

Libref  = WORK
Table   = home_equity_compute_py
Dsopts  = {}
Results = Pandas

In [33]:
sascode='proc print data=home_equity_compute_py(obs=10);run;'
sassub=sas.submitLST(sascode,method='listandlog')
#sassub=sas.submitLOG(sascode)
#sassub=sas.submit(sascode)
#print(results[‘LOG’]) HTML(results[‘LST’])




Obs,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc,appdate,city,state,division,region,diff,loanstatus
1,1,1100,25860.00,39025.00,HomeImp,Other,10.5000,0.00000,0.00000,94.367,1.00000,9.0000,33.7799,22040,oregon,Wisconsin,East North Central,Midwest,-13165.00,Default
2,1,1300,70053.00,68400.00,HomeImp,Other,7.0000,0.00000,2.00000,121.833,0.00000,14.0000,33.7799,22229,churchton,Maryland,South Atlantic,South,1653.00,Default
3,1,1500,13500.00,16700.00,HomeImp,Other,4.0000,0.00000,0.00000,149.467,1.00000,10.0000,33.7799,21396,orcas,Washington,Pacific,West,-3200.00,Default
4,1,1500,73760.82,101776.05,DebtCon,Other,8.9223,0.25457,0.44944,179.766,1.18606,21.2961,33.7799,22213,hastings,Florida,South Atlantic,South,-28015.23,Default
5,0,1700,97800.00,112000.00,HomeImp,Office,3.0000,0.00000,0.00000,93.333,0.00000,14.0000,33.7799,22329,wilmington,California,Pacific,West,-14200.00,Repaid
6,1,1700,30548.00,40320.00,HomeImp,Other,9.0000,0.00000,0.00000,101.466,1.00000,8.0000,37.1136,21534,olympia,Washington,Pacific,West,-9772.00,Default
7,1,1800,48649.00,57037.00,HomeImp,Other,5.0000,3.00000,2.00000,77.100,1.00000,17.0000,33.7799,21936,williston,Florida,South Atlantic,South,-8388.00,Default
8,1,1800,28502.00,43034.00,HomeImp,Other,11.0000,0.00000,0.00000,88.766,0.00000,8.0000,36.8849,21804,longview,Washington,Pacific,West,-14532.00,Default
9,1,2000,32700.00,46740.00,HomeImp,Other,3.0000,0.00000,2.00000,216.933,1.00000,12.0000,33.7799,21724,elma,Washington,Pacific,West,-14040.00,Default
10,1,2000,73760.82,62250.00,HomeImp,Sales,16.0000,0.00000,0.00000,115.800,0.00000,13.0000,33.7799,21713,montclair,New Jersey,Middle Atlantic,Northeast,11510.82,Default


119284  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
119284! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml3.htm
119285  
119286  proc print data=home_equity_compute_py(obs=10);run;

NOTE: There were 10 observations read from the data set WORK.HOME_EQUITY_COMPUTE_PY.
NOTE: PROCEDURE PRINT used (Total process time):
      real time           0.02 seconds
      cpu time            0.04 seconds
      

119287  ods html5 (id=saspy_internal) close;ods listing;
119288  




In [ ]:
## Create a string with SAS code to connect to the CAS server, delete the global table if it exists, and then load data to CAS
## NOTE: You can also use the Python SWAT package to accomplish the same tasks below using all Python.

prepareLoadingToCAS = '''
* Connect to the CAS Server *;
cas conn;

* Drop and load data to the CAS server *;
proc casutil;
	* Drop the global scope CAS table if it exists *;
	droptable casdata='home_equity_cas_py' incaslib="casuser" quiet;

	* Send the SAS data set to the CAS server and promote the table *;
	load data=work.home_equity_compute_py casout="home_equity_cas_py" outcaslib="casuser" promote;

	* View in-memory CAS tables *;
	list tables;
quit;
'''

# Submit the above SAS code
SAS.submit(prepareLoadingToCAS)